## Summarization

we try to summarize on the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset .

In [1]:
!pip install sentencepiece
!pip install 'transformers[torch]'
!pip install datasets
!pip install zstandard
!pip install sacrebleu
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=73f14d2b717a179258b124734bb4a1ef2e50b2e0fc3dc33534fe166db17e541b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
from datasets import load_dataset, DatasetDict
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from datasets import load_metric
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
ds = load_dataset("cnn_dailymail", '1.0.0', split="train")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
train_testvalid = ds.train_test_split(train_size=1000, test_size=200) #https://huggingface.co/docs/datasets/v2.18.0/en/package_reference/main_classes#datasets.Dataset.train_test_split
train_testvalid

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 200
    })
})

In [5]:
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 100
    })
    valid: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 100
    })
})

In [7]:
ds["train"][1]

{'article': "These are the shocking pictures of a Roma teenager who was beaten and left for dead in a supermarket trolley. Vigilantes kidnapped the 16-year-old boy, known only as Darius, from the camp he shares with his family near the northern commuter town of Pierrefitte-sur-Seine on the outskirts of Paris. The thugs rang the youngster's mother demanding a €12,000 ransom for the safe return of her son. The figure was later reduced to €4,000, but when it was not paid, the gang battered Darius before dumping him in a shopping trolley before leaving it on the roadside. The teenager, known only as Darius, left, was found dumped in a shopping trolley by members of his family, right, after he was brutally assaulted by the vigilante gang on the outskirts of Paris on Friday night . Doctors in a Paris hospital have placed Darius in a medically induced coma following Friday's brutal assault, which has left him fighting for his life. Police confirmed that Darius was set upon on Friday night by 

In [8]:
sample_text = ds["train"][1]["article"][:2000]
sample_text


"These are the shocking pictures of a Roma teenager who was beaten and left for dead in a supermarket trolley. Vigilantes kidnapped the 16-year-old boy, known only as Darius, from the camp he shares with his family near the northern commuter town of Pierrefitte-sur-Seine on the outskirts of Paris. The thugs rang the youngster's mother demanding a €12,000 ransom for the safe return of her son. The figure was later reduced to €4,000, but when it was not paid, the gang battered Darius before dumping him in a shopping trolley before leaving it on the roadside. The teenager, known only as Darius, left, was found dumped in a shopping trolley by members of his family, right, after he was brutally assaulted by the vigilante gang on the outskirts of Paris on Friday night . Doctors in a Paris hospital have placed Darius in a medically induced coma following Friday's brutal assault, which has left him fighting for his life. Police confirmed that Darius was set upon on Friday night by a ‘dozen mas

## Todo
Implement a baseline summary. An easy way to generate a baseline summary of a newspaper article is to take the first few sentences only. Write a function to generate a baseline summary given the article text.

HINT: We have imported the sent_tokenize from nltk.

In [9]:
def get_baseline_summary(text):
    #TODO: Write a function to generate a baseline summary
    sentences = sent_tokenize(text)
    #summary = sentences[0] + " " + sentences[1] + " " + sentences[2]
    summary = " ".join(sentences[0:2])
    return summary

In [10]:
get_baseline_summary(ds["train"][1]["article"])

'These are the shocking pictures of a Roma teenager who was beaten and left for dead in a supermarket trolley. Vigilantes kidnapped the 16-year-old boy, known only as Darius, from the camp he shares with his family near the northern commuter town of Pierrefitte-sur-Seine on the outskirts of Paris.'

In [11]:
summaries = {}

In [12]:
summaries["baseline"] = get_baseline_summary(sample_text)

### GPT2
Can implement a summarization by using the text-generation pipeline but appending a "TL:DR" at the end of the prompt

In [13]:
pipe = pipeline("text-generation", model="gpt2") #If gpt2-xl is too large use a small version
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))
summaries["gpt2"]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"\nThe perpetrators left the boy with his mother inside the trash bin in order to steal his car, before being followed by gardaí on Saturday morning outside the site of the alleged assault.\nDarius and his mother face an immediate punishment of three years' imprisonment and seven years' imprisonment with the following three sentences:\n\n• A three-year"

In [14]:
import pprint
pprint.pprint(summaries["gpt2"])

('\n'
 'The perpetrators left the boy with his mother inside the trash bin in order '
 'to steal his car, before being followed by gardaí on Saturday morning '
 'outside the site of the alleged assault.\n'
 "Darius and his mother face an immediate punishment of three years' "
 "imprisonment and seven years' imprisonment with the following three "
 'sentences:\n'
 '\n'
 '• A three-year')


### T5

In [15]:
pipe = pipeline("summarization", model="t5-small") #If t5-large is too large use t5-small version
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
summaries["t5"]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


'teenager, known only as darius, was found dumped in a shopping trolley on the outskirts of Paris .\nthe 16-year-old was brutally assaulted by the vigilante gang on friday night .\nhis mother said she received a call from vigilantes on his abandoned mobile phone .\nhe was held in basement by men wearing masks and carrying weapons .'

## BART

In [16]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
summaries["bart"]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

"Vigilantes kidnapped the 16-year-old boy, known only as Darius, from the camp he shares with his family near the northern commuter town of Pierrefitte-sur-Seine.\nThe thugs rang the youngster's mother demanding a €12,000 ransom for the safe return of her son.\nThe figure was later reduced to €4,000, but when it was not paid, the gang battered Darius before dumping him in a shopping trolley.\nDoctors in a Paris hospital have placed Darius in a medically induced coma following Friday's brutal assault."

In [17]:
print("GROUND TRUTH")
print(ds["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Assault happened after boy, 16, accused of stealing in Pierrefitte-sur-Seine . He was discovered in a supermarket trolley near the camp where he lived . Boy's mother tells detectives vigilantes asked her for £12,000 . The demand was later reduced to £4,000, but was not paid . President Hollande describes attack as 'unspeakable and unjustifiable' Doctors have placed the youngster in a medically induced coma . WARNING GRAPHIC CONTENT .

BASELINE
These are the shocking pictures of a Roma teenager who was beaten and left for dead in a supermarket trolley. Vigilantes kidnapped the 16-year-old boy, known only as Darius, from the camp he shares with his family near the northern commuter town of Pierrefitte-sur-Seine on the outskirts of Paris.

GPT2

The perpetrators left the boy with his mother inside the trash bin in order to steal his car, before being followed by gardaí on Saturday morning outside the site of the alleged assault.
Darius and his mother face an immediate punishm

## Evaluation

How can we evaluate the performance of a summarization model?

**BLEU (Bilingual Evaluation Understudy) Score**  is a precision based metric that measures how many words or n-grams present in the machine generated summaries also appear in the human reference summaries divided by the length of the generation. It penalises the appearance of too short summaries with a brevity penalty. [Sacrebleu](https://huggingface.co/spaces/evaluate-metric/sacrebleu). Sacrebleu bleu score goes from 0-100.

**ROUGE (Recall Oriented Understudy for Gisting Evaluation) score** is the recall based metric that measures how many words/ngrams present in the human reference summaries were found in the machine generated summaries. Rouge score goes from 0-1. Some sub-forms:

1. ROUGE-N: Overlap of n-grams
2. ROUGE-1: Overlap of unigrams.
3. ROUGE-2: Oerlap of bigrams.
4. ROUGE-L: Overlap of the Longest Common Subsequence

To Do:
1. How do you interpret a Bleu/ Rouge Score of 0?

In [18]:
bleu_metric = load_metric("sacrebleu")

<ipython-input-18-a80436835c78>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [19]:
rouge_metric = load_metric("rouge")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [20]:
reference = ds["train"][1]["highlights"]
reference

"Assault happened after boy, 16, accused of stealing in Pierrefitte-sur-Seine . He was discovered in a supermarket trolley near the camp where he lived . Boy's mother tells detectives vigilantes asked her for £12,000 . The demand was later reduced to £4,000, but was not paid . President Hollande describes attack as 'unspeakable and unjustifiable' Doctors have placed the youngster in a medically induced coma . WARNING GRAPHIC CONTENT ."

In [21]:
bleu_scores = []
for model_name in summaries:
  bleu_metric.add(prediction=summaries[model_name], reference=[reference])
  results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
  bleu_scores.append(results["score"])
  results["precisions"] = [np.round(p, 2) for p in results["precisions"]]

In [25]:
results

{'score': 11.695287083133337,
 'counts': [41, 17, 6, 3],
 'totals': [92, 91, 90, 89],
 'precisions': [44.57, 18.68, 6.67, 3.37],
 'bp': 1.0,
 'sys_len': 92,
 'ref_len': 73}

In [22]:
bleu_scores

[5.43261998010243, 0.0, 0.0, 11.695287083133337]

In [23]:
summaries

{'baseline': 'These are the shocking pictures of a Roma teenager who was beaten and left for dead in a supermarket trolley. Vigilantes kidnapped the 16-year-old boy, known only as Darius, from the camp he shares with his family near the northern commuter town of Pierrefitte-sur-Seine on the outskirts of Paris.',
 'gpt2': "\nThe perpetrators left the boy with his mother inside the trash bin in order to steal his car, before being followed by gardaí on Saturday morning outside the site of the alleged assault.\nDarius and his mother face an immediate punishment of three years' imprisonment and seven years' imprisonment with the following three sentences:\n\n• A three-year",
 't5': 'teenager, known only as darius, was found dumped in a shopping trolley on the outskirts of Paris .\nthe 16-year-old was brutally assaulted by the vigilante gang on friday night .\nhis mother said she received a call from vigilantes on his abandoned mobile phone .\nhe was held in basement by men wearing masks an

In [24]:

records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    rouge_score = rouge_metric.compute()
    result_dict = dict((rn, rouge_score[rn].mid.fmeasure) for rn in rouge_names)
    bleu_metric.add(prediction=summaries[model_name], reference=[reference])
    bleu_results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
    result_dict["bleu"] = bleu_results["score"]
    records.append(result_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum,bleu
baseline,0.360656,0.116667,0.180328,0.180328,5.432620
gpt2,0.174603,0.000000,0.126984,0.142857,0.000000
t5,0.285714,0.032258,0.142857,0.142857,0.000000
bart,0.553459,0.267516,0.364780,0.389937,11.695287


## Finetuning a model for Summarization

#Discussion
How would you fine tune a model for summarization?

In [51]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [52]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [53]:
def convert_data_to_features(batch, tokenizer):
    input_encodings = tokenizer(batch["article"], max_length=1024,
                                truncation=True)

    with tokenizer.as_target_tokenizer(): #as_target_tokenizer helps to differentiate between
        target_encodings = tokenizer(batch["highlights"], max_length=128,
                                     truncation=True)

    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]}

In [54]:
dataset_cnn = ds.map(convert_data_to_features, batched=True, fn_kwargs={"tokenizer": tokenizer})
columns = ["input_ids", "labels", "attention_mask"]
dataset_cnn.set_format(type="torch", columns=columns)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [55]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [56]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='output_folder', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10, push_to_hub=False,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16)

In [57]:
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_cnn["train"],
                  eval_dataset=dataset_cnn["valid"])

In [58]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=62, training_loss=3.1290874173564296, metrics={'train_runtime': 131.3648, 'train_samples_per_second': 7.612, 'train_steps_per_second': 0.472, 'total_flos': 450089709189120.0, 'train_loss': 3.1290874173564296, 'epoch': 0.99})

In [59]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

In [68]:
pipe = pipeline("summarization", model = model, tokenizer = tokenizer)
pipe(ds["train"][1]["article"])

[{'summary_text': "Brits have named Beyonce as the celebrity they'd most like to travel with, according to Icelolly ."}]

In [60]:
rouge_score = evaluate_summaries(ds["test"], rouge_metric, trainer.model, tokenizer,batch_size=2)

100%|██████████| 50/50 [00:45<00:00,  1.09it/s]


In [61]:
rouge_dict = dict((rn, rouge_score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"cnn"])

,rouge1,rouge2,rougeL,rougeLsum
cnn,0.277179,0.106216,0.205069,0.204232


## Todo
1. Compare another model (check to see if there is a finetuned model) against these metrics.
2. How would you calculate the BLEU score?

# Reading list

1. [Bertscore](https://huggingface.co/spaces/evaluate-metric/bertscore) uses pre-trained contextual embeddings from BERT and matches words in candidate and reference sentences by cosine similarity.
2. An article on the [BLEU](https://www.traceloop.com/blog/demystifying-the-bleu-metric) metric.